In [34]:
import requests
import json
import pandas as pd
import time
import random
import xmltodict
import os


In [23]:
conf = open('config.json','r') 
config = json.load(conf)
api_key = config['API_key']
conf.close()
keyw = open('keywords.txt','r')
keywords = keyw.readlines()
keyw.close()
for index in range(len(keywords)):
    keywords[index] = keywords[index].replace('\n','')
print(keywords)
Affiliation = 'New Zealand'
start = 0
count = 25
step = 25
url = "https://api.elsevier.com/content/search/scopus"
headers=  {'X-ELS-APIKey':api_key}
print(url)


['clean technology', 'clean-energy', 'carbon emissions', 'energy efficient', 'green energy', 'renewable energy', 'smart home', 'geothermal', 'wastewater treatment', 'carbon emissions', 'zero waste', 'energy storage', 'heat recovery', 'green building', 'additive manufacturing', 'waste reduction', 'maritime electrification']
https://api.elsevier.com/content/search/scopus


In [24]:
def QueryEdit(url,keyword,Affiliation=None,daterange=None,start = 0,sort = 'citedby-count', count = 25):
    url +='?query={}'.format(keyword)
    if Affiliation!=None:
        url+="+AFFIL({})".format(Affiliation)
    url += "&count={}".format(count)
    url += "&sort={}".format(sort)
    url += "&start={}".format(start)
    return url

In [ ]:
def TidyJsontoDf(web_json,paper_info_short,column):
    try :
        for paper in web_json['search-results']['entry']:
            try:
                title = paper['dc:title']
                autor = paper['dc:creator']
                doi = paper['prism:doi']
                citedby_count = paper['citedby-count']
                affilname = paper['affiliation'][0]['affilname']
                affiliation_city = paper['affiliation'][0]['affiliation-city']
                affiliation_country = paper['affiliation'][0]['affiliation-country']
            except:
                continue
            paper_data = [[title,autor,doi,citedby_count,affilname,affiliation_city,affiliation_country]]
            paper_info = pd.DataFrame(data=paper_data, columns=column)
            paper_info_short = pd.concat([paper_info_short,paper_info],ignore_index=True)
    except:
        pass
    return paper_info_short

In [31]:
def GetAndTidy(keyword,Affiliation = None,url=url, headers = headers,limit = 5000):
    url = QueryEdit(url,keyword,Affiliation = Affiliation,count = count)
    print(url)
    web = requests.get(url,headers=headers)
    result = web.text
    web_json = json.loads(result)
    print(web_json['search-results']['opensearch:totalResults'])
    epos = int(web_json['search-results']['opensearch:totalResults']) // step +1
    column = ['title','autor','doi','citedby_count','affilname','affiliation-city','affiliation-country']
    paper_info_short = pd.DataFrame(columns=column)
    paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
    for i in range(1,epos+1):
        start = i*25
        if start > limit: break
        url = "https://api.elsevier.com/content/search/scopus"
        url = QueryEdit(url,keyword,Affiliation = Affiliation,start = start,count = count)
        print(url)
        web = requests.get(url,headers=headers)
        if(web.ok):
            result = web.text
            web_json = json.loads(result)
            paper_info_short = TidyJsontoDf(web_json,paper_info_short,column)
        # time.sleep(random.random())
    return paper_info_short



### Run one keyword

In [ ]:
keyword = 'geothermal'
url = "https://api.elsevier.com/content/search/scopus"
paper_info = GetAndTidy(keyword,url = url)
paper_info.info()
filepath = 'data/'
paper_info.to_csv(filepath+keyword+'.csv',index=False)

### Run all keywords

In [ ]:
filepath = 'data/'
for keyword in keywords:
    url = "https://api.elsevier.com/content/search/scopus"
    paper_info = GetAndTidy(keyword,url = url)
    paper_info.to_csv(filepath+keyword+'.csv',index=False)

### NZ Data

In [33]:
filepath = 'data/NZ/'
for keyword in keywords:
    url = "https://api.elsevier.com/content/search/scopus"
    paper_info = GetAndTidy(keyword,Affiliation='New Zealand',limit=500,url = url)
    paper_info.to_csv(filepath+keyword+'_NZ.csv',index=False)

https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-count&start=0
2623
https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-count&start=25
https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-count&start=50
https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-count&start=75
https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-count&start=100
https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-count&start=125
https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-count&start=150
https://api.elsevier.com/content/search/scopus?query=clean technology+AFFIL(New Zealand)&count=25&sort=citedby-

In [49]:
# clean nz data
filepath = 'data/NZ/'
filelist = os.listdir(filepath)
print(filelist)
for file in filelist:
    df = pd.read_csv(filepath + file)
    selected_data = df.query(" `affiliation-country` == 'New Zealand' ")
    selected_data.to_csv(filepath+file,index=False)



['energy storage_NZ.csv', 'energy efficient_NZ.csv', 'clean-energy_NZ.csv', 'waste reduction_NZ.csv', 'wastewater treatment_NZ.csv', 'green energy_NZ.csv', 'renewable energy_NZ.csv', 'clean technology_NZ.csv', 'maritime electrification_NZ.csv', 'green building_NZ.csv', 'zero waste_NZ.csv', 'smart home_NZ.csv', 'carbon emissions_NZ.csv', 'heat recovery_NZ.csv', 'additive manufacturing_NZ.csv', 'geothermal_NZ.csv']


,title,autor,doi,citedby_count,affilname,affiliation-city,affiliation-country
0,Convection in porous media,Nield D.A.,10.1007/978-1-4614-5541-7,6347,The University of Auckland,Auckland,New Zealand
6,A review on phase change energy storage: Mater...,Farid M.M.,10.1016/j.enconman.2003.09.015,2416,The University of Auckland,Auckland,New Zealand
9,A review on energy conservation in building ap...,Khudhair A.,10.1016/S0196-8904(03)00131-6,1122,The University of Auckland,Auckland,New Zealand
11,Coalbed methane: A review,Moore T.A.,10.1016/j.coal.2012.05.011,1050,University of Canterbury,Christchurch,New Zealand
13,A comparative assessment of factors which infl...,Wardle D.A.,10.1111/j.1469-185X.1992.tb00728.x,973,AgResearch Ruakura,Hamilton,New Zealand
...,...,...,...,...,...,...,...
505,Landscape memory: The imprint of the past on c...,Brierley G.J.,10.1111/j.1475-4762.2009.00900.x,93,The University of Auckland,Auckland,New Zealand
507,From feeding one to feeding many: Hormone-indu...,Augustine R.,10.1113/jphysiol.2007.146316,93,School of Biomedical Sciences,Dunedin,New Zealand
510,Studying Drugs in Human Milk: Time to Unify th...,Begg E.J.,10.1177/089033402237904,93,Christchurch Hospital New Zealand,Christchurch,New Zealand
511,Shade effects on stream periphyton and inverte...,Quinn J.,10.1080/00288330.1997.9516797,93,National Institute of Water and Atmospheric Re...,Auckland,New Zealand


### TEST

In [ ]:
url = "https://api.elsevier.com/content/search/scopus"
keyword = 'additive manufacturing'
url = QueryEdit(url,keyword,Affiliation = None,count = 25)
print(url)
web = requests.get(url,headers=headers)
result = web.text
web_json = json.loads(result)
print(web_json)